In [108]:
import import_ipynb
import copy
from builtins import range
import cubiecube as QBC
import corner as Co
import edge as Ed
import facelet as Fc
import facecube as FC
import numpy as np
import coordinatecube as Coord
from scipy import special

In [118]:
class PhaseOneCoordinateCube(Coord.CoordinateCube):
    def __init__(self, cp = 0, co = 0, ep_phase1 = 0, eo = 0, **kwarg):
        """
        As a subclass of CoordinateCube, TwoPhaseCoordinateCube also expresses a cube pattern in numbers. 
        ep_UDSlice_phase1 indicates the permutation of middle layer edges in the phase 1.
        ep_UDSlice_phase2 indicates the permutation of middle layer edges in the phase 2.
        Also, ep_others represents the permutation of other edges.
        
        """
        if kwarg['CubieCube'] is not None:
            super().__init__(CubieCube = kwarg['CubieCube'])
            self.ep_UDSlice_phase1 = self.get_UDSlice(kwarg['CubieCube'])
        else: 
            self.cp = cp
            self.co = co
            self.eo = eo
            self.ep_UDSlice_phase1 = ep_phase1

    def get_UDSlice(self, QB):    
        """
        This method returns the corresponding ep_UDSlice_phase1 of the patern QB, which is expressed in CubieCube.
        
        """
        # get the distribution of middle layer edges
        position_of_UDSlice = [0 for i in range(12)]
        for i in range(12):
            if QB.ep[i] == Ed.FR or QB.ep[i] == Ed.FL or QB.ep[i] == Ed.BL or QB.ep[i] == Ed.BR:
                position_of_UDSlice[i] = 1
        ep_phase1 = 0
        ct = -1
        reach_the_leftmost = False
        for i in range(12):
            if position_of_UDSlice[i] == 1:
                reach_the_leftmost = True
                ct += 1
                continue
            if reach_the_leftmost:
                ep_phase1 += sp.special.comb(i, ct, exact = True)
        return ep_phase1
    
    def to_CubieCube(self):
        # idea: find a tuple (x1, x2, x3, x4) to record the gap between the edges and the right 
        # Note that x1 + x2 + x3 + x4 <= 8
        # 8 - (x1 + x2 + x3 + x4) represents the position of the leftmost edge
        # 8 - (x2 + x3 + x4) + 1 is the second; 8 - (x3 + x4) + 1 + 1 the third; 8 - x4 + 1 + 1 + 1 the forth.
        if kwarg['CubieCube'] is not None:
            return kwarg['CubieCube']
    
        position_of_UDSlice = [0 for i in range(12)]
        n = self.ep_UDSlice_phase1
        if n <= 8:               # x2, x3, x4 must be 0
            position_of_UDSlice[-1] = 1
            position_of_UDSlice[-2] = 1
            position_of_UDSlice[-3] = 1
            position_of_UDSlice[8 - n] = 1

        elif 9 <= n <= 44:       # x3, x4 must be 0
            position_of_UDSlice[-1] = 1
            position_of_UDSlice[-2] = 1
            found = False
            for x1 in range(9):
                for x2 in range(8 - x1 + 1):
                    # sum up in the defining way and check whether it matches n
                    if x1 + sum([8 - x2 + 1 + 1 + i for i in range(x2)]) == n :
                        found = True
                        break
                if found:
                    position_of_UDSlice[8 - x2 + 1] = 1
                    position_of_UDSlice[8 - (x1 + x2)] = 1
                    break

        elif 45 <= n <= 164:     # x4 must be 0
            position_of_UDSlice[-1] = 1
            found = False
            for x1 in range(9):
                for x2 in range(8 - x1 + 1):
                    for x3 in range(8 - x2 - x1 + 1):
                        # sum up in the defining way and check whether it matches n
                        if x1 + sum([special.comb(8 - (x2 + x3) + 1 + 1 + i, 1 ) for i in range(x2)]) + np.sum(np.array([special.comb(8 - (x3) + 1 + 1 + 1 + i, 2) for i in range(x3)])) == n :
                            found = True
                            break
                    if found:
                        break
                if found:
                    position_of_UDSlice[8 - (x1 + x2 + x3)] = 1
                    position_of_UDSlice[8 - (x2 + x3 ) + 1] = 1
                    position_of_UDSlice[8 - x3 + 1 + 1] = 1    
                    break

        elif 165 <= n <= 494:
            found = False
            for x1 in range(9):
                for x2 in range(8 - x1 + 1):
                    for x3 in range(8 - x2 - x1 + 1):
                        for x4 in range(8 - x3 - x2 - x1 + 1):
                            # sum up in the defining way and check whether it matches n
                            if x1 + sum([special.comb(8 - (x2 + x3 + x4) + 1 + 1 + i, 1 ) for i in range(x2)]) + np.sum(np.array([special.comb(8 - (x3 + x4) + 1 + 1 + 1 + i, 2) for i in range(x3)])) + np.sum(np.array([special.comb(8 - (x4) + 1 + 1 + 1 + 1 + i, 3) for i in range(x4)]))  == n :
                                found = True
                                break
                        if found:
                            break
                    if found:
                        break
                if found:
                    position_of_UDSlice[8 - (x1 + x2 + x3 + x4)] = 1
                    position_of_UDSlice[8 - (x2 + x3 + x4) + 1] = 1
                    position_of_UDSlice[8 - (x3 + x4) + 1 + 1] = 1    
                    position_of_UDSlice[8 - x4 + 1 + 1 + 1] = 1
                    break
        else:
            print('out of range')
            
        # We choose a arbitrary representative of the corresponding coset.
        edge_perm = [i for i in range(12)]
        middle_layer = 8    # FR, FL, BL, BR 
        for i in range(12):
            if position_of_UDSlice[i] == 1:
                edge_perm[i] = middle_layer
                edge_perm[middle_layer] = i
                middle_layer += 1 
        ep_coor = 0
        for i in range(1, len(edge_perm)):
            # count the the number of all edges left of XX, whose orders are higher than the order of XX.
            count = 0
            for j in range(i): 
                if edge_perm[j] > edge_perm[i]:
                    count += 1
            ep_coor += count * np.math.factorial(i)
        Q = Coord.CoordinateCube(self.cp, self.co, ep_coor, self.eo, CubieCube = None)
        
        return Q.to_cubiecube()

In [119]:
class PhaseTwoCoordinateCube(Coord.CoordinateCube):
    def __init__(self, cp, co, ep_phase2, ep_UDSlice_phase2, eo, **kwarg):
        """
        In phase 2, the cube is in the subgroup G1, in which UDSlice edges are in the middle layer.
        Thus we can describe edge permutation with the one of middle layer and the one of edge permutation
        """
        if kwarg['CubieCube'] is not None:
            super().__init__(CubieCube = kwarg['CubieCube'])
            self.ep_UDSlice_phase2 = self.get_ep_UDSlice_phase2(kwarg['CubieCube'])
            self.ep_phase2 = self.get_ep_phase2(kwarg['CubieCube'])
        else: 
            self.cp = cp
            self.co = co
            self.eo = eo
            self.ep_UDSlice_phase2 = ep_phase1
            self.ep_phase2 = ep_phase2
    
    def get_ep_UDSlice_phase2(self, QB):
        ep_UDSlice = QB.ep[8:]
        ep_coor = 0
        for i in range(1, len(ep_UDSlice)):
            # count the the number of all edges left of XX, whose orders are higher than the order of XX.
            count = 0
            for j in range(i): 
                if ep_UDSlice[j] > ep_UDSlice[i]:
                    count += 1
            ep_coor += count * np.math.factorial(i)
        return ep_coor
    
    def get_ep_phase2(self, QB):
        ep_others = QB.ep[:8]
        ep_coor = 0
        for i in range(1, len(ep_others)):
            # count the the number of all edges left of XX, whose orders are higher than the order of XX.
            count = 0
            for j in range(i): 
                if ep_others[j] > ep_others[i]:
                    count += 1
            ep_coor += count * np.math.factorial(i)
        return ep_coor